In [1]:
import sklearn
import corus
import pandas as pd
import numpy as np
import nltk
import json
import tqdm

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import string
import re

from pprint import pprint

from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/partnadem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/partnadem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
MAX_LENTA_DATASET_ROWS = 10000
RANDOM_STATE = 5555
NUM_TOPICS = 10
CHUNK_SIZE = 750

#### Загрузить набор данных Lenta.ru с помощью пакета Corus

In [3]:
!pip install requests

In [4]:
import requests
import os

files = os.listdir()


def download_file(url, name):
    response = requests.get(url)
    open(name, "wb").write(response.content)

In [5]:
import requests
from corus import load_lenta2

url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2"

path = 'lenta-ru-news.csv.bz2'

download_file(url, path)
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [6]:
wnl = WordNetLemmatizer()
STOP_WORDS = stopwords.words('russian')

download_file(
    "https://raw.githubusercontent.com/danilkladnitsky/nlp-tasks/hw_5/hw_5/stopwords-ru.json", "stopwords-ru.json")
download_file(
    "https://raw.githubusercontent.com/danilkladnitsky/nlp-tasks/hw_5/hw_5/stopwords-ru.txt", "stopwords-ru.txt")


external_stop_word_1 = open(
    "stopwords-ru.txt", "r", encoding="utf-8").read().replace('\n', ' ').split(" ")

raw_json = open(
    "stopwords-ru.json", "r", encoding="utf-8").read()
external_stop_word_2 = json.loads(raw_json)

external_stop_words = set(external_stop_word_1 + external_stop_word_2)

STOP_WORDS.extend(external_stop_words)


def lemmatize_token(token): return wnl.lemmatize(token)


def normalize_text(text):
    split_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(
        '', '', string.punctuation)).lower().split()
    result = ' '.join([lemmatize_token(word)
                      for word in split_text if word not in STOP_WORDS])

    return result

<>:26: DeprecationWarning: invalid escape sequence \w
<>:26: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_652/3383728445.py:26: DeprecationWarning: invalid escape sequence \w
  split_text = re.sub('\w*\d\w*', '', text).translate(str.maketrans(


In [7]:
def load_lenta_to_list(path, max_number=None):
    records = load_lenta2(path)
    texts, titles, dates, original_topics, raw_content = [], [], [], [], []
    for i, record in enumerate(records):
        texts.append(normalize_text(record.text))
        titles.append(record.title)
        dates.append(record.date)
        original_topics.append(record.topic)
        raw_content.append(record.text)

        if not max_number is None:
            if i >= max_number-1:
                break
    result = {}
    result['topic'] = titles
    result['date'] = dates
    result['original_topics'] = original_topics
    result['content'] = texts
    result['raw_content'] = raw_content

    return result

In [8]:
lenta_list = load_lenta_to_list(path, max_number=MAX_LENTA_DATASET_ROWS)

lentaFrame = pd.DataFrame(lenta_list)
lentaFrame

,topic,date,original_topics,content,raw_content
0,1914. Русские войска вступили в пределы Венгрии,1914-09-16,Библиотека,бои сопоцкина друскеник закончились отступлени...,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,1914-09-16,Библиотека,министерство народного просвещения виду происх...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,1914-09-17,Библиотека,штабскапитан п н нестеров днях увидев районе ж...,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,1914-09-17,Библиотека,фотографкорреспондент daily mirror рассказывае...,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,1914-09-18,Библиотека,лица приехавшие варшаву люблина передают доста...,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...,...,...,...,...
9995,На границе Дагестана и Чечни будут стрелять в ...,2000-07-10,Россия,глава мвд дагестана приказал нарядам милиции п...,Глава МВД Дагестана приказал нарядам милиции н...
9996,Путин пересадит губернаторов в Госсовет,2000-07-10,Россия,представители кремля ведут активный диалог губ...,Представители Кремля ведут активный диалог с г...
9997,В Москву съедутся клоуны со всего мира,2000-07-10,Россия,москве июля открывается фестиваль планета клоу...,"В Москве 20 июля открывается фестиваль ""Планет..."
9998,Старшая Уильямс выигрывает чемпионский титул,2000-07-10,Спорт,чисто американским получился финал уимблдонско...,Чисто американским получился финал Уимблдонск...


#### Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…)

In [9]:
tokenized_content = lentaFrame.content.apply(lambda x: x.split())
id2word = corpora.Dictionary(tokenized_content)

In [10]:
corpus = [id2word.doc2bow(text) for text in tokenized_content]
corpus_example = [[(id2word[id], freq) for id, freq in cp]
                  for cp in corpus[:2]]
corpus_example[0][:50]

[('«русский', 1),
 ('австрийцев', 1),
 ('автомобили', 1),
 ('артиллерийском', 1),
 ('артиллерийскую', 1),
 ('артиллерию', 1),
 ('батареей', 1),
 ('бежала', 1),
 ('безуспешными', 1),
 ('ближе', 1),
 ('бои', 1),
 ('большая', 1),
 ('борьбу', 1),
 ('бою', 1),
 ('бросив', 1),
 ('венгрии', 1),
 ('взяли', 1),
 ('вступили', 1),
 ('вылазки', 1),
 ('высот', 1),
 ('галиции', 1),
 ('гарнизона', 1),
 ('германской', 1),
 ('германцев', 1),
 ('дембицу', 1),
 ('достиг', 1),
 ('друскеник', 1),
 ('закончились', 1),
 ('заняли', 1),
 ('захватываются', 1),
 ('значительного', 1),
 ('инвалид»', 1),
 ('калибры', 1),
 ('колонна', 1),
 ('крепости', 1),
 ('крепостью', 1),
 ('материальная', 1),
 ('напряжения', 1),
 ('начал', 1),
 ('нашей', 1),
 ('неприятель', 1),
 ('неприятельский', 1),
 ('новые', 1),
 ('обнаруживается', 1),
 ('обоз', 1),
 ('обстреливалась', 1),
 ('огонь', 1),
 ('орудия', 1),
 ('осовцу', 1),
 ('остаются', 1)]

In [11]:
def get_coherence(corpus, topics, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           random_state=RANDOM_STATE,
                                           chunksize=CHUNK_SIZE,
                                           num_topics=topics,
                                           alpha=a,
                                           eta=b,
                                           workers=3)

    coherence_model_lda = CoherenceModel(
        model=lda_model, texts=tokenized_content, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [12]:
%%time

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=NUM_TOPICS,
                                       random_state=RANDOM_STATE,
                                       chunksize=CHUNK_SIZE,
                                       alpha='symmetric',
                                       per_word_topics=True,
                                       workers=3)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"россии" + 0.004*"сообщает" + 0.004*"словам" + 0.003*"чечне" + '
  '0.003*"новости" + 0.003*"боевиков" + 0.003*"риа" + 0.002*"заявил" + '
  '0.002*"президента" + 0.002*"сообщил"'),
 (1,
  '0.008*"россии" + 0.004*"заявил" + 0.004*"сообщает" + 0.004*"словам" + '
  '0.003*"президента" + 0.003*"сообщил" + 0.002*"рф" + 0.002*"чечни" + '
  '0.002*"новости" + 0.002*"президент"'),
 (2,
  '0.006*"россии" + 0.005*"новости" + 0.005*"риа" + 0.004*"сообщили" + '
  '0.004*"мвд" + 0.003*"сообщает" + 0.003*"боевиков" + 0.003*"районе" + '
  '0.003*"словам" + 0.003*"данным"'),
 (3,
  '0.004*"сообщает" + 0.003*"рф" + 0.003*"россии" + 0.003*"новости" + '
  '0.003*"риа" + 0.003*"словам" + 0.003*"рублей" + 0.003*"заявил" + '
  '0.002*"президента" + 0.002*"сша"'),
 (4,
  '0.005*"сообщает" + 0.003*"россии" + 0.003*"компании" + 0.003*"словам" + '
  '0.003*"долларов" + 0.003*"риа" + 0.003*"заявил" + 0.002*"новости" + '
  '0.002*"чемпионата" + 0.002*"передает"'),
 (5,
  '0.006*"россии" + 0.005*"риа

#### Визуализировать результаты работы LDA с помощью pyLDAvis

In [13]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.011724 -0.007305       1        1  17.049845
5      0.032784 -0.014521       2        1  14.424989
2      0.077328  0.025765       3        1  10.884404
7     -0.013085 -0.006510       4        1  10.849098
6     -0.023317  0.021576       5        1   9.612846
4     -0.032272  0.048049       6        1   8.535139
3     -0.032898 -0.003389       7        1   8.508064
9     -0.006676 -0.000219       8        1   8.170550
1     -0.008823 -0.052315       9        1   8.109743
0      0.018682 -0.011130      10        1   3.855322, topic_info=               Term         Freq        Total Category  logprob  loglift
471          россии  5216.000000  5216.000000  Default  30.0000  30.0000
1369       боевиков  1099.000000  1099.000000  Default  29.0000  29.0000
1506          чечне  1328.000000  1328.000000  Default  28.0000  28.0000
998        сообщили  1382.000000  1382.000000  Default  27.0000  27.0000
4548         рублей  1163.000000  1163.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
608        долларов    55.462635  2058.912709  Topic10  -6.5550  -0.3585
652   правительства    48.937031  1223.018680  Topic10  -6.6802   0.0372
2061            сша    55.631678  2290.016321  Topic10  -6.5520  -0.4618
521          данным    50.844388  1602.567302  Topic10  -6.6420  -0.1949
1709       компании    47.679099  1335.011933  Topic10  -6.7062  -0.0765

[917 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
7127       5  0.910699      atlantis
29203      7  0.043042            bt
29203      8  0.903872            bt
61450      1  0.047333          card
61450      2  0.757320          card
...      ...       ...           ...
36101      6  0.022731  ястржембский
36101      7  0.045461  ястржембский
36101      8  0.022731  ястржембский
36101      9  0.227306  ястржембский
36101     10  0.176162  ястржембский

[3259 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 3, 8, 7, 5, 4, 10, 2, 1])

#### Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей

In [14]:
get_coherence(corpus, NUM_TOPICS, 1, 1)

0.3582489865201352

In [15]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 3
max_topics = 12
step_size = 3
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                 }

if 1 == 0:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha) *
                     len(topics_range)))

    for k in topics_range:
        for a in alpha:
            for b in beta:
                cv = get_coherence(corpus=corpus,
                                   topics=k, a=a, b=b)

                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)

                pbar.update(1)

    pd.DataFrame(model_results).to_csv(
        './lda_tuning_results.csv', index=False)
    pbar.close()

#### Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик

In [16]:
!pip install hyperopt

In [17]:
import artm
from sklearn.feature_extraction.text import CountVectorizer
import os
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"  # protobuf fix

/home/partnadem/.pyenv/versions/3.9.12/envs/wsl-3.9.12/lib/python3.9/site-packages/future/standard_library/__init__.py:65: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [18]:
tokenized_content = lentaFrame.content.to_list()

cv = CountVectorizer(max_features=1000, max_df=0.8,
                     min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(tokenized_content).todense()).T

print(n_wd.shape)

(1000, 10000)


In [19]:
token_list = [i for i in cv.vocabulary_.keys()]
bv = artm.BatchVectorizer(data_format='bow_n_wd',
                          n_wd=n_wd, vocabulary=token_list)

/home/partnadem/.pyenv/versions/3.9.12/envs/wsl-3.9.12/lib/python3.9/site-packages/artm/batches_utils.py:227: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [20]:
def get_artm_model(seed=1000):
    model = artm.ARTM(num_topics=40, dictionary=bv.dictionary,
                      cache_theta=True, seed=seed)
    model.scores.add(artm.PerplexityScore(
        name='perplexity_score', dictionary=bv.dictionary))

    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(
        name='top_tokens_score', num_tokens=100))

    model.fit_offline(bv, num_collection_passes=30)
    return model

In [21]:
%%time

model = get_artm_model()
model.score_tracker["perplexity_score"].last_value

CPU times: user 35.8 s, sys: 1.69 s, total: 37.5 s
Wall time: 8.3 s


239.1787872314453

In [22]:
def get_model_instance(context):
    model = artm.ARTM(
        num_topics=context['num_topics'], dictionary=bv.dictionary, cache_theta=True)

    model.scores.add(artm.PerplexityScore(
        name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(
        name='top_tokens_score', num_tokens=10))

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
            name='SparsePhi',
            tau=context['phi_tau']
        ),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=context['theta_tau']
        ),
    )
    model.regularizers.add(
        artm.DecorrelatorPhiRegularizer(
            name='DecorrelatorPhi',
            tau=context['decorrelation_tau']
        ),
    )

    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    print("Perplexity:", perplexity_score)
    return {'loss': perplexity_score, 'status': STATUS_OK}

In [23]:
context = {
    'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
    'theta_tau': hp.uniform('SparseTheta', -1, 1),
    'phi_tau': hp.uniform('SparsePhi', -1, 1),
    'num_topics': hp.choice('num_topics', [3, 6, 9, 12, 15]),
}

trials = Trials()

result = fmin(
    fn=get_model_instance,
    space=context,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials,
)

print(result)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

Perplexity:                                           
419.776611328125                                      
Perplexity:                                                                   
547.4457397460938                                                             
Perplexity:                                                                   
348.8782958984375                                                             
Perplexity:                                                                    
487.32366943359375                                                             
Perplexity:                                                                    
403.5289611816406                                                              
Perplexity:                                                                    
380.49285888671875                                                             
Perplexity:                                                                    
452.0794067382

#### Winner
{'DecorrelatorPhi': 3106.636084605295, 'SparsePhi': 0.44741088069669166, 'SparseTheta': 0.005391870879176919, 'num_topics': 15}

In [ ]:
best_settings = {'DecorrelatorPhi': 3106.636084605295, 'SparsePhi': 0.44741088069669166,
                 'SparseTheta': 0.005391870879176919, 'num_topics': 4}

model = artm.ARTM(num_topics=15,
                  dictionary=bv.dictionary, cache_theta=True, seed=seed)
model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                      dictionary=bv.dictionary))

model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))

model.regularizers.add(artm.DecorrelatorPhiRegularizer(
    name='DecorrelatorPhi', tau=best_settings['DecorrelatorPhi']))
model.regularizers.add(artm.SmoothSparsePhiRegularizer(
    name='SparsePhi', tau=best_settings['SparsePhi']))
model.regularizers.add(artm.SmoothSparseThetaRegularizer(
    name='SparseTheta', tau=best_settings['SparseTheta']))


model.fit_offline(bv, num_collection_passes=30)


model.score_tracker["perplexity_score"].last_value